In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import optuna

In [2]:
%run -i classes.ipynb

In [ ]:
# Manual hyperparameters
epochs = 50
patience = 7
num_trials = 1 # For hyperparameter tuning

In [3]:
# Import and split the data
data = pd.read_csv('./data/train.csv')
X = data.drop('loan_status', axis=1)
y = data['loan_status']

preprocessor = LoanPreprocessor()
X_processed = preprocessor.fit_transform(X)

X_train, X_temp, y_train, y_temp = train_test_split(
    X_processed, y, test_size=0.3, random_state=420
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=69
)

train_dataset = LoanDataset(X_train, y_train.values)
val_dataset = LoanDataset(X_val, y_val.values)
test_dataset = LoanDataset(X_test, y_test.values)

In [4]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
# Create an Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=num_trials)

# Print the best hyperparameters
print("Best hyperparameters: ", study.best_params)
print("Best accuracy: ", study.best_value)

# Create a DataFrame to store the results
results = []
for trial in study.trials:
    results.append({
        'Trial': trial.number,
        'Hidden Size 1': trial.params['hidden_size1'],
        'Hidden Size 2': trial.params['hidden_size2'],
        'Dropout Rate': trial.params['dropout_rate'],
        'Learning Rate': trial.params['learning_rate'],
        'Batch Size': trial.params['batch_size'],
        'Accuracy': trial.value
    })

results_df = pd.DataFrame(results)

# Display the results
print("\nHyperparameter Tuning Results:")
print(results_df)

# Optionally, save the results to a CSV file
results_df.to_csv('hyperparameter_tuning_results.csv', index=False)

[I 2024-12-20 05:09:37,693] A new study created in memory with name: no-name-57ec2030-4620-4a5c-afed-3ee278f5c55e


Epoch [1/50], Training Loss: 0.4464, Validation Loss: 0.3070
Best model weights saved.
Epoch [2/50], Training Loss: 0.2739, Validation Loss: 0.2511
Best model weights saved.
Epoch [3/50], Training Loss: 0.2441, Validation Loss: 0.2378
Best model weights saved.
Epoch [4/50], Training Loss: 0.2345, Validation Loss: 0.2324
Best model weights saved.
Epoch [5/50], Training Loss: 0.2291, Validation Loss: 0.2282
Best model weights saved.
Epoch [6/50], Training Loss: 0.2246, Validation Loss: 0.2253
Best model weights saved.
Epoch [7/50], Training Loss: 0.2205, Validation Loss: 0.2216
Best model weights saved.
Epoch [8/50], Training Loss: 0.2165, Validation Loss: 0.2187
Best model weights saved.
Epoch [9/50], Training Loss: 0.2127, Validation Loss: 0.2163
Best model weights saved.
Epoch [10/50], Training Loss: 0.2094, Validation Loss: 0.2127
Best model weights saved.
Epoch [11/50], Training Loss: 0.2062, Validation Loss: 0.2103
Best model weights saved.
Epoch [12/50], Training Loss: 0.2034, Val

[I 2024-12-20 05:11:42,738] Trial 0 finished with value: 0.9420256905763328 and parameters: {'hidden_size1': 43, 'hidden_size2': 59, 'dropout_rate': 0.23474574438861484, 'learning_rate': 5.2383623894552675e-05, 'batch_size': 26}. Best is trial 0 with value: 0.9420256905763328.


Best model weights saved to best_model_trial_0.pth
Best hyperparameters:  {'hidden_size1': 43, 'hidden_size2': 59, 'dropout_rate': 0.23474574438861484, 'learning_rate': 5.2383623894552675e-05, 'batch_size': 26}
Best accuracy:  0.9420256905763328

Hyperparameter Tuning Results:
   Trial  Hidden Size 1  Hidden Size 2  Dropout Rate  Learning Rate  \
0      0             43             59      0.234746       0.000052   

   Batch Size  Accuracy  
0          26  0.942026  


In [6]:
# Load the best model weights
best_model_weights = torch.load(f"best_model_trial_{study.best_trial.number}.pth")

preprocessor = LoanPreprocessor()

# Load the best model
best_model = LoanNeuralNetwork(
    input_size=X_processed.shape[1], 
    hidden_size1=study.best_params['hidden_size1'], 
    hidden_size2=study.best_params['hidden_size2'], 
    dropout_rate=study.best_params['dropout_rate']
).to(device)

# Load the best model weights
best_model.load_state_dict(best_model_weights)

criterion = nn.BCELoss()

In [7]:
best_model.eval()
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)
test_loss = validate_model(best_model, test_loader, criterion, device)

correct = 0
total = 0
with torch.no_grad():
    for features, labels in test_loader:
            features, labels = features.to(device), labels.to(device)
            outputs = best_model(features)
            predicted = (outputs.view(-1) > 0.5).float()  # Ensure outputs are flattened
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_accuracy = correct / total
print(f'Test Loss: {test_loss:.4f}')
print(f'Test accuracy: {test_accuracy:.4f}')

Test Loss: 0.1840
Test accuracy: 0.9421


In [9]:
# Train on all data with best hyperparameters
data = pd.read_csv('./data/train.csv')
X = data.drop('loan_status', axis=1)
y = data['loan_status']

preprocessor = LoanPreprocessor()
X_processed = preprocessor.fit_transform(X)

final_dataset = LoanDataset(X_processed, y.values)

# Train the final model
final_model = LoanNeuralNetwork(
    input_size=X_processed.shape[1], 
    hidden_size1=study.best_params['hidden_size1'], 
    hidden_size2=study.best_params['hidden_size2'], 
    dropout_rate=study.best_params['dropout_rate']
).to(device)

final_loader = DataLoader(final_dataset, batch_size=study.best_params['batch_size'], shuffle=True)
optimizer = optim.Adam(final_model.parameters(), lr=study.best_params['learning_rate'])

final_model.train()
early_stopping = EarlyStopping(patience=7, verbose=True)
best_loss = float('inf')
best_model_weights = None  # To store the best model weights

for epoch in range(epochs):
    total_loss = 0
    for features, labels in final_loader:
        features, labels = features.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = final_model(features)
        loss = criterion(outputs.view(-1), labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(final_loader)
    print(f'Epoch [{epoch+1}/{epochs}], Training Loss: {avg_loss:.4f}')

    # Check early stopping
    early_stopping(avg_loss)
    if early_stopping.early_stop:
        print("Training stopped early.")
        break

    # Save the best model
    if avg_loss < best_loss:
        best_loss = avg_loss
        best_model_weights = final_model.state_dict()  # Save the model weights
        print("Best model weights saved.")

# Save the best model weights to a file
final_model_save_path = f"final_model.pth"
torch.save(best_model_weights, final_model_save_path)
print(f"Best model weights saved to {final_model_save_path}")

Epoch [1/5], Training Loss: 0.4119
Best model weights saved.
Epoch [2/5], Training Loss: 0.2679
Best model weights saved.
Epoch [3/5], Training Loss: 0.2476
Best model weights saved.
Epoch [4/5], Training Loss: 0.2386
Best model weights saved.
Epoch [5/5], Training Loss: 0.2333
Best model weights saved.
Best model weights saved to final_model.pth


In [10]:
# Load the test data for submission
raw_submission_data = pd.read_csv('./data/test.csv')
submission_data = preprocessor.fit_transform(raw_submission_data)

# Create the test dataset
submission_dataset = LoanDataset(submission_data, np.zeros(len(submission_data)))  # Placeholder for labels

# Create the test data loader
submission_loader = DataLoader(submission_dataset, batch_size=study.best_params['batch_size'], shuffle=False)


In [11]:
# Make predictions on the test data
predictions = []
final_model.eval()
with torch.no_grad():
    for features, _ in submission_loader:  # Ignore labels in test data
        features = features.to(device)
        outputs = final_model(features)
        predicted = (outputs.view(-1) > 0.5).float()  # Ensure outputs are flattened
        predictions.extend(predicted.cpu().numpy())

# Create a DataFrame to store the predictions
predictions_df = pd.DataFrame({
    'id': raw_submission_data['id'],  # Include the original 'id' column
    'loan_status': predictions
})

# Save the predictions to a CSV file
predictions_df.to_csv('test_predictions.csv', index=False)

print("Test predictions saved to test_predictions.csv")

Test predictions saved to test_predictions.csv
